# Aula 06: Apriori
Integrantes:

    1) RM357067 - Edinaldo Rodrigues de Oliveira Junior
    2) RM358158 - Henrique Cardoso
    3) RM358067 - Jefferson de Souza Santos
    4) RM357344 - Walace Vinicius Silva dos Santos

In [8]:
import pandas as pd 

In [9]:
sleep = pd.read_csv('/Users/jeffsantos/Desktop/9DTSR/git/9DTSR/Machine Learning/Trabalhinhos/Trabalho Aula 06/sleep.csv')
sleep.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [25]:
pd.get_dummies(sleep['Sleep Disorder'])

,Insomnia,Sleep Apnea
0,False,False
1,False,False
2,False,False
3,False,True
4,False,True
...,...,...
369,False,True
370,False,True
371,False,True
372,False,True


In [27]:
import numpy as np 
gender = pd.get_dummies(sleep['Gender']) 
sleep['Age_2'] = np.select([(sleep['Age'] <= 18),
                            (sleep['Age'] <= 25),
                            (sleep['Age'] <= 50),
                            (sleep['Age'] > 50)], 
                            ['adolescente', 'jovem', 'adulto', 'idoso'], default='desconhecido')

age = pd.get_dummies(sleep['Age_2'])

occu = pd.get_dummies(sleep['Occupation'])

qual = pd.get_dummies(sleep['Quality of Sleep'])
qual.columns = ['qualidade_4', 'qualidade_5', 'qualidade_6', 'qualidade_7', 'qualidade_8', 'qualidade_9']

str = pd.get_dummies(sleep['Stress Level'])
str.columns = ['stress_3', 'stress_4', 'stress_5', 'stress_6', 'stress_7', 'stress_8']

bmi = pd.get_dummies(sleep['BMI Category'])

dis = pd.get_dummies(sleep['Sleep Disorder'])


sleep_tratado = pd.concat([gender,age,occu, qual, bmi, str, dis], axis=1)


sleep_tratado

,Female,Male,adulto,idoso,Accountant,Doctor,Engineer,Lawyer,Manager,Nurse,...,Obese,Overweight,stress_3,stress_4,stress_5,stress_6,stress_7,stress_8,Insomnia,Sleep Apnea
0,False,True,True,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
1,False,True,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,True,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,True,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True
4,False,True,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,True,False,False,True,False,False,False,False,False,True,...,False,True,True,False,False,False,False,False,False,True
370,True,False,False,True,False,False,False,False,False,True,...,False,True,True,False,False,False,False,False,False,True
371,True,False,False,True,False,False,False,False,False,True,...,False,True,True,False,False,False,False,False,False,True
372,True,False,False,True,False,False,False,False,False,True,...,False,True,True,False,False,False,False,False,False,True


In [33]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

regras = apriori(sleep_tratado, min_support=0.10, use_colnames=True)
df = association_rules(regras, 
                  metric='confidence',
                  min_threshold=0.50)

df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Female),(adulto),0.494652,0.796791,0.291444,0.589189,0.739452,1.0,-0.102691,0.494652,-0.410811,0.291444,-1.021622,0.477481
1,(idoso),(Female),0.203209,0.494652,0.203209,1.000000,2.021622,1.0,0.102691,inf,0.634228,0.410811,1.000000,0.705405
2,(Nurse),(Female),0.195187,0.494652,0.195187,1.000000,2.021622,1.0,0.098637,inf,0.627907,0.394595,1.000000,0.697297
3,(qualidade_9),(Female),0.189840,0.494652,0.184492,0.971831,1.964675,1.0,0.090587,17.939840,0.606065,0.368984,0.944258,0.672402
4,(Female),(Overweight),0.494652,0.395722,0.283422,0.572973,1.447918,1.0,0.087678,1.415082,0.612159,0.466960,0.293327,0.644595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,"(stress_5, Male)","(Lawyer, Normal, qualidade_8, adulto)",0.173797,0.112299,0.112299,0.646154,5.753846,1.0,0.092782,2.508719,1.000000,0.646154,0.601390,0.823077
622,"(stress_5, Normal)","(Lawyer, qualidade_8, adulto, Male)",0.155080,0.112299,0.112299,0.724138,6.448276,1.0,0.094884,3.217914,1.000000,0.724138,0.689240,0.862069
623,"(stress_5, adulto)","(Lawyer, Normal, qualidade_8, Male)",0.179144,0.112299,0.112299,0.626866,5.582090,1.0,0.092182,2.379037,1.000000,0.626866,0.579662,0.813433
624,(Lawyer),"(qualidade_8, stress_5, Male, Normal, adulto)",0.125668,0.155080,0.112299,0.893617,5.762289,1.0,0.092811,7.942246,0.945245,0.666667,0.874091,0.808877


In [35]:
df[df['consequents'] == frozenset({'qualidade_8'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
44,(Lawyer),(qualidade_8),0.125668,0.291444,0.112299,0.893617,3.066172,1.0,0.075674,6.660428,0.770715,0.368421,0.849859,0.639469
56,(Normal),(qualidade_8),0.521390,0.291444,0.262032,0.502564,1.724394,1.0,0.110076,1.424417,0.877722,0.475728,0.297958,0.700823
58,(stress_4),(qualidade_8),0.187166,0.291444,0.114973,0.614286,2.107733,1.0,0.060425,1.836997,0.646573,0.316176,0.455633,0.504391
59,(stress_5),(qualidade_8),0.179144,0.291444,0.160428,0.895522,3.072710,1.0,0.108217,6.781895,0.821770,0.517241,0.852549,0.722991
142,"(Lawyer, Male)",(qualidade_8),0.120321,0.291444,0.112299,0.933333,3.202446,1.0,0.077233,10.628342,0.781806,0.375000,0.905912,0.659327
154,"(Normal, Male)",(qualidade_8),0.350267,0.291444,0.176471,0.503817,1.728692,1.0,0.074387,1.428013,0.648772,0.379310,0.299726,0.554661
158,"(stress_5, Male)",(qualidade_8),0.173797,0.291444,0.160428,0.923077,3.167255,1.0,0.109776,9.211230,0.828209,0.526316,0.891437,0.736768
169,"(Lawyer, adulto)",(qualidade_8),0.125668,0.291444,0.112299,0.893617,3.066172,1.0,0.075674,6.660428,0.770715,0.368421,0.849859,0.639469
189,"(Normal, adulto)",(qualidade_8),0.438503,0.291444,0.262032,0.597561,2.050347,1.0,0.134233,1.760655,0.912342,0.560000,0.432029,0.748322
194,"(adulto, stress_4)",(qualidade_8),0.181818,0.291444,0.114973,0.632353,2.169725,1.0,0.061983,1.927273,0.658915,0.320896,0.481132,0.513424


In [30]:
df[df['consequents'] == frozenset({'qualidade_9'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
37,(idoso),(qualidade_9),0.203209,0.18984,0.176471,0.868421,4.574500,1.0,0.137894,6.157219,0.980679,0.814815,0.837589,0.898999
62,(stress_3),(qualidade_9),0.189840,0.18984,0.184492,0.971831,5.119222,1.0,0.148453,28.760695,0.993208,0.945205,0.965230,0.971831
77,"(idoso, Female)",(qualidade_9),0.203209,0.18984,0.176471,0.868421,4.574500,1.0,0.137894,6.157219,0.980679,0.814815,0.837589,0.898999
100,"(Female, stress_3)",(qualidade_9),0.179144,0.18984,0.179144,1.000000,5.267606,1.0,0.145136,inf,0.986971,0.943662,1.000000,0.971831
215,"(idoso, stress_3)",(qualidade_9),0.171123,0.18984,0.171123,1.000000,5.267606,1.0,0.138637,inf,0.977419,0.901408,1.000000,0.950704
251,"(idoso, Female, stress_3)",(qualidade_9),0.171123,0.18984,0.171123,1.000000,5.267606,1.0,0.138637,inf,0.977419,0.901408,1.000000,0.950704


In [36]:
df[df['consequents'] == frozenset({'Insomnia'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
69,(stress_7),(Insomnia),0.133690,0.205882,0.109626,0.820000,3.982857,1.0,0.082101,4.411765,0.864499,0.476744,0.773333,0.676234
212,"(adulto, Overweight)",(Insomnia),0.283422,0.205882,0.155080,0.547170,2.657682,1.0,0.096729,1.753676,0.870432,0.464000,0.429769,0.650208
248,"(stress_7, Overweight)",(Insomnia),0.117647,0.205882,0.101604,0.863636,4.194805,1.0,0.077383,5.823529,0.863158,0.457831,0.828283,0.678571


In [37]:
df[df['consequents'] == frozenset({'Sleep Apnea'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
50,(Nurse),(Sleep Apnea),0.195187,0.208556,0.163102,0.835616,4.006674,1.0,0.122394,4.814617,0.932411,0.677778,0.792299,0.808834
94,"(Nurse, Female)",(Sleep Apnea),0.195187,0.208556,0.163102,0.835616,4.006674,1.0,0.122394,4.814617,0.932411,0.677778,0.792299,0.808834
106,"(Female, Overweight)",(Sleep Apnea),0.283422,0.208556,0.168449,0.594340,2.849782,1.0,0.109340,1.951001,0.905828,0.520661,0.487443,0.701016
233,"(Nurse, Overweight)",(Sleep Apnea),0.176471,0.208556,0.157754,0.893939,4.286325,1.0,0.120950,7.462185,0.930993,0.694118,0.865991,0.825175
263,"(Nurse, Female, Overweight)",(Sleep Apnea),0.176471,0.208556,0.157754,0.893939,4.286325,1.0,0.120950,7.462185,0.930993,0.694118,0.865991,0.825175


# Conclusão

- Apesar da base não apresentar regras com nível de suporte muito alto, conseguimos com ela ver a aplicação do modelo 
- Para participantes da pesquisa com Insomnia observamos que 10% possui um nível alto de stress igual a 8 com condinaça de 82%. Overwight também aparece em duas associações.
- Para Apnea observamos que participação da profissão de enfermeiro, sexo feminino e também sobrepeso.
- já para o índice de qualidade de sono a categoria idoso e biaxo nível de stress (3) se sobresaem.